In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train = pd.read_csv("train_feateng")
test = pd.read_csv("test_feateng")

In [26]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,day,hour
0,3567540,0,16.554679,3.663562,0,7.497762,5.081404,5.010635,1,4.762174,...,0,31,0,0,0,0,2,19,3,2
1,3567541,0,16.554681,3.432373,0,9.439625,5.590987,5.010635,0,5.111988,...,0,31,0,0,0,0,2,19,3,2
2,3567542,0,16.554683,4.799914,0,9.714745,6.165418,5.010635,0,4.836282,...,0,31,0,0,0,0,2,19,3,2
3,3567543,0,16.554686,5.141664,0,8.969669,6.202536,5.010635,0,5.420535,...,0,31,0,0,0,0,2,19,3,2
4,3567544,0,16.554687,4.681668,0,9.028219,6.276643,5.010635,1,5.411646,...,0,31,0,0,0,0,2,19,3,2


In [27]:
X = train.drop(['isFraud'], axis=1)
y = train['isFraud']


In [28]:

for feature in X.columns:
    X[feature] = pd.to_numeric(X[feature])

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [30]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBClassifier 
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
}


In [31]:
# Make a basic XGBoost Classifier to feed into RandomSearchCV
classifier = XGBClassifier()

In [32]:
# Create our random_search model
rand_search = RandomizedSearchCV(classifier, param_distributions=params, n_iter=5, scoring='roc_auc', n_jobs=-1, cv=5, verbose=3)

In [33]:
# Now fit our parameters 
rand_search.fit(X_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  3.4min finished


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           reg_lambda=None,
                                           scale_pos_weight=None,
                                           subsample=No

In [36]:
# Now we create a model based on the BEST hyperparameters
best_classifier = rand_search.best_estimator_
best_classifier

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, gamma=0.0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=15,
              min_child_weight=7, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [37]:
y_pred = best_classifier.predict(X_test)

In [38]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.9968602825745683

In [39]:
# Now Compare y_pred with y_test
# No hyperparam tuning: 0.027076282218172352
# With hyperparam tuning: 0.0031241864097891173
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, y_pred)


0.0031241864097891173

In [40]:
# No hyperparam tuning: 0.9713392391870767
# With hyperparam tuning: 1.0
recall_score(y_test, y_pred)

1.0

In [41]:
# No hyperparam tuning: 0.974385781495034
# With hyperparam tuning: 0.9938208032955715
precision_score(y_test, y_pred)

0.9938208032955715